In [1]:
from particletracking import dataframes
import filehandling

In [11]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
flat_dir = "/media/data/Data/FirstOrder/Susceptibility/Flat2"
dimpled_dir = "/media/data/Data/FirstOrder/Susceptibility/Dimpled2"

In [4]:
flat_files = filehandling.get_directory_filenames(flat_dir+'/*.hdf5')
dimpled_files = filehandling.get_directory_filenames(dimpled_dir+'/*.hdf5')

In [5]:
## Squared and abs badly

def get_psis(df, param):
    df = df.loc[df.edge_distance > 3*df.r.mean()]
    return np.abs(np.mean(df[param].values)**2), np.abs(np.mean(df[param].values))

def get_susceptibility(df, param):
    psi = np.array(df.groupby('frame').apply(get_psis, param).values)
    psi_2, psi = zip(*psi)
    sus = np.mean(psi_2) - np.mean(psi)**2
    return sus

In [22]:
def get_psis(df, param):
    """Return psi_a^2 and psi_a"""
    df = df.loc[df.edge_distance > 3*df.r.mean()]
    vals = df[param].values
    return np.mean(vals)*np.conj(np.mean(vals)), np.mean(vals)

def get_susceptibility(df, param):
    psi = np.array(df.groupby('frame').apply(get_psis, param).values)
    psi_2, psi = zip(*psi)
    sus = np.real(np.mean(psi_2) - np.mean(psi)*np.conj(np.mean(psi)))
    return sus

In [6]:
def get_suses(files, param, G):
    duties = []
    order_suses = []
    torder_suses = []
    for file in tqdm(files):
        data = dataframes.DataStore(file)
        data.df['order_long_complex'] = data.df.order_r_long + 1j*data.df.order_i_long
        data.df = add_torder(data.df, G)
        duties.append(data.df.Duty.mean().round())
        order_suses.append(get_susceptibility(data.df, param))
        torder_suses.append(get_susceptibility(data.df, 'torder'))
    return duties, order_suses, torder_suses
            

In [7]:
def guess_G_length(points):
    r = points.r.mean()
    large_spacing = 4.8/2*r
    apothem = large_spacing / (2*np.sqrt(3)/3)
    return 2*np.pi / apothem

def guess_G(points):
    L = guess_G_length(points)
    angles = np.arange(1, 120) * np.pi / 180
    Gs = np.stack((np.cos(angles), np.sin(angles)), axis=0)*L
    return Gs

def add_torder(df, G):
    df['torder'] = np.exp(1j*df[['x', 'y']].values@G)
    return df

In [8]:
def get_G(df):
    points = df.loc[0]
    G_all = guess_G(points)
    torder_all = np.array([np.exp(1j*points[['x', 'y']]@G) for G in G_all.T])
    G = G_all[:, np.argmax(np.sum(torder_all, axis=1))]
    return G

In [19]:
G_flat = get_G(dataframes.DataStore(flat_files[-1]).df)
flat_duties, flat_order_suses, flat_torder_suses = get_suses(flat_files, 'order_long_complex', G_flat)


100%|██████████| 100/100 [07:22<00:00,  4.43s/it]


In [23]:
G_dimpled = get_G(dataframes.DataStore(dimpled_files[-1]).df)
dimple_duties, dimple_order_suses, dimple_torder_suses = get_suses(dimpled_files, 'order_long_complex', G_dimpled)

100%|██████████| 100/100 [07:23<00:00,  4.43s/it]


In [25]:
fig, ax = plt.subplots()
ax.set_title('Flat 3r high density')
ax.plot(flat_duties, flat_order_suses, 'r-')
ax.set_xlabel('Duty')
ax2 = ax.twinx()
ax2.plot(flat_duties, flat_torder_suses, 'g-')
ax.set_ylabel('$\chi_6$', c='r')
ax2.set_ylabel('$\chi_T$', c='g')

Text(0, 0.5, '$\\chi_T$')

In [24]:
fig, ax = plt.subplots()
ax.set_title('Dimple 3r high density')
ax.plot(dimple_duties, dimple_order_suses, 'r-')
ax2 = ax.twinx()
ax.set_xlabel('Duty')
ax2.plot(dimple_duties, dimple_torder_suses, 'g-')
ax.set_ylabel('$\chi_6$', c='r')
ax2.set_ylabel('$\chi_T$', c='g')
plt.tight_layout()

In [88]:
import pandas as pd

In [75]:
sus_data = pd.DataFrame({
    'duty': flat_duties, 
    'dimple_order_sus': dimple_order_suses, 
    'dimple_torder_sus': dimple_torder_suses,
    'flat_order_sus': flat_order_suses,
    'flat_torder_sus': flat_torder_suses})

In [76]:
sus_data.to_csv("/media/data/Data/FirstOrder/Susceptibility/sus_data_middle.csv")